<a href="https://colab.research.google.com/github/Chinjushibu/godot/blob/master/AI-detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai_detector


In [17]:
from detector import OpenaiDetector

sentence     = """All children, except one, grow up. They soon know that they will grow up, and the way Wendy knew was this. One day when she was two years old she was playing in a garden, and she plucked another flower and ran with it to her mother. I suppose she must have looked rather delightful, for Mrs. Darling put her hand to her heart and cried, “Oh, why can’t you remain like this for ever!” This was all that passed between them on the subject, but henceforth Wendy knew that she must grow up. You always know after you are two. Two is the beginning of the end. Of course they lived at 14, and until Wendy came her mother was the chief one. She was a lovely lady, with a romantic mind and such a sweet mocking mouth. Her romantic mind was like the tiny boxes, one within the other, that come from the puzzling East, however many you discover there is always one more; and her sweet mocking mouth had one kiss on it that Wendy could never get, though there it was, perfectly conspicuous in the right-hand corner. The way Mr. Darling won her was this: the many gentlemen who had been boys when she was a girl discovered simultaneously that they loved her, and they all ran to her house to propose to her except Mr. Darling, who took a cab and nipped in first, and so he got her. He got all of her, except the innermost box and the kiss. He never knew about the box, and in time he gave up trying for the kiss. Wendy thought Napoleon could have got it, but I can picture him trying, and then going off in a passion, slamming the door. Mr. Darling used to boast to Wendy that her mother not only loved him but respected him. He was one of those deep ones who know about stocks and shares. Of course no one really knows, but he quite seemed to know, and he often said stocks were up and shares were down in a way that would have made any woman respect him."""
bearer_token = 'Bearer sess-abcd1234..'

od = OpenaiDetector(bearer_token)
response = od.detect(sentence)
print(response)

### Output

{"output"          : "The classifier considers the text to be very unlikely AI-generated.",
"Confidence score" : 96.15870427207666}

Check prompt, Length of sentence it should be more than 1,000 characters


{'output': 'The classifier considers the text to be very unlikely AI-generated.',
 'Confidence score': 96.15870427207666}

In [18]:
import requests
import numpy as np


In [19]:
class OpenaiDetector:

    def __init__(self, token):

        self.header = {
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
            'Authorization': token,
            'Connection': 'keep-alive',
            'Content-Type': 'application/json',
            'Origin': 'https://platform.openai.com',
            'Referer': 'https://platform.openai.com/',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
        }

        self.possible_classes = ['very unlikely', 'unlikely', 'unclear if it is', 'possibly', 'likely']
        self.class_max = [10, 45, 90, 98, 99]

    def detect(self, text, all_probs=False):
        data = {
            'prompt': text + "».\n<|disc_score|>",
            'max_tokens': 1,
            'temperature': 1,
            'top_p': 1,
            'n': 1,
            'logprobs': 5,
            'stop': '\n',
            'stream': False,
            'model': 'model-detect-v2',
        }

        response = requests.post('https://api.openai.com/v1/completions', headers=self.header, json=data)
        if response.status_code == 200:
            choices = response.json()['choices'][0]
            logprobs = choices['logprobs']['top_logprobs'][0]
            probs = {key: 100 * np.e ** value for key, value in logprobs.items()}
            key_prob = probs['"']
            if self.class_max[0] < key_prob < self.class_max[len(self.class_max) - 1]:
                val = max(i for i in self.class_max if i < key_prob)
                class_label = self.possible_classes[self.class_max.index(val)]
            elif self.class_max[0] > key_prob:
                class_label = self.possible_classes[0]
            else:
                class_label = self.possible_classes[len(self.possible_classes) - 1]
            top_prob = {'Class': class_label, 'AI-Generated Probability': key_prob}
            if all_probs:
                return probs, top_prob
            return top_prob
        return "Check prompt, Length of sentence it should be more than 1,000 characters"

"""Python setup.py for package"""

In [20]:
import io
import os

from setuptools import find_packages, setup


In [23]:
def read(*paths, **kwargs):
    content = ""
    with io.open(
        os.path.join(os.path.dirname(__file__), *paths),
        encoding=kwargs.get("encoding", "utf8"),
    ) as open_file:
        content = open_file.read().strip()
    return content


In [22]:
def read_requirements(path):
    return [line.strip() for line in read(path).split("\n") if not line.startswith(('"', "#", "-", "git+"))]


setup(
    name="openai-detector",
    version="0.1.2",
    description="Open AI classifier for indicating AI-written text",
    url="https://github.com/promptslab/openai-detector",
    long_description=read("README.md"),
    long_description_content_type="text/markdown",
    license="Apache",
    author="monk1337",
    maintainer="All our contributors",
    packages=find_packages(),
    install_requires=["requests","numpy"],
    python_requires=">=3.7.0",)

NameError: ignored